# 0.   Packages and Functions

In [1]:
# !pip install jupyter_nbextensions_configurator

In [2]:
# !jupyter nbextensions_configurator enable --user

In [3]:
 !jupyter nbextension enable nbextensions_configurator/config_menu/main

Enabling notebook extension nbextensions_configurator/config_menu/main...
      - Validating: problems found:
        - require?  X nbextensions_configurator/config_menu/main


In [4]:
!pip install pandas_datareader

In [5]:

#general
import pandas as pd
import numpy as np
import pickle 
from decimal import Decimal
import re


#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [6]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [7]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [8]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [9]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [10]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'NVDA',
             #automobiles
            'F',
            'GM',
            'HMC',
            'TSLA',
            'TYO',
             'HOG',
             #retail
             'WMT',
             'M',
             'TGT',
             'KSS',
             'COST',
             'BBY',
             #FastMovingConsumerGoods(FMCG)
             'NESN',
             'KO',
             'PEP',
             'ULVR',
             'PG',
             'OR',
            #bank
            'AXP',
            'VISA',
            'SC',
            'DB',
            'UBS',
            'BAC',
            'JPM']

In [11]:
tech_c=['AAPL','AMZN','MSFT','GOOG','FB','VISA','NVDA']
auto_c=['F','GM','HMC','TSLA','TYO','HOG']
retail_c=[ 'WMT', 'M','TGT','KSS','COST','BBY']
fmcg_c=['NESN','KO','PEP','ULVR','PG', 'OR']
bank_c=['AXP','VISA','SC', 'DB', 'UBS','BAC','JPM']

In [12]:
#ESG ratios
ESG_ratios=[16.5, 27.4, 14.7, 22.8, 31.6, 12.9, 29.7, 30.6, 28.5, 31.1, 30.3, 16.3, 27.3, 14.5, 14.8, 12.9, 22.3, 12.0, 28.3, 25.1, 17.6, 23.4, 25.1, 20.5, 19.8, 17.4, 26.9, 30.0, 24.8, 26.3, 27.9]

We are going to extract info from 10 years from the companies selected

In [13]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()
error_indexes=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)
        
        error_company_index=companies.index(company)
        error_indexes.append(error_company_index)


for c in error_companies:
  companies.remove(c)

ESG_ratios_fix=list()

for index in range(0, len(ESG_ratios)):
    if index not in error_indexes:
        ESG_ratios_fix.append(ESG_ratios[index])

AAPL
AMZN
MSFT
GOOG
FB
NVDA
F
GM
HMC
TSLA
TYO
HOG
WMT
M
TGT
KSS
COST
BBY
There were  errors when extracting data of   NESN
KO
PEP
There were  errors when extracting data of   ULVR
PG
OR
AXP
There were  errors when extracting data of   VISA
SC
DB
UBS
BAC
JPM


In [14]:
companies

['AAPL',
 'AMZN',
 'MSFT',
 'GOOG',
 'FB',
 'NVDA',
 'F',
 'GM',
 'HMC',
 'TSLA',
 'TYO',
 'HOG',
 'WMT',
 'M',
 'TGT',
 'KSS',
 'COST',
 'BBY',
 'KO',
 'PEP',
 'PG',
 'OR',
 'AXP',
 'SC',
 'DB',
 'UBS',
 'BAC',
 'JPM']

In [15]:
len(ESG_ratios_fix)

28

In [16]:
len(companies)

28

In [17]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [18]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

# 3. Random transactions

In [19]:
transactions=pd.DataFrame()
#we do 500.000 initial transactions

#id
transactions['id']=range(1,500001)

## Companies

In [20]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Sector

In [21]:
#sector of the company

random_companies=transactions['company'].tolist()
random_sectors=list()
for comp in random_companies:
    if comp in tech_c:
        sector='TECH'
    elif comp in auto_c:
        sector='AUTO'
    elif comp in retail_c:
        sector='RETAIL'
    elif comp in fmcg_c:
        sector='FMCG'
    elif comp in bank_c:
        sector='BANK'
    
    random_sectors.append(sector)

transactions['sector']=random_sectors


## Horizons

In [22]:
#random horizon
transactions["horizon (days)"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [23]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [24]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

500000


### Fixing dates (BUY)

In [25]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [26]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY


In [ ]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [ ]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

## Expected return and volatility

### Expected volatilities

In [ ]:

l_volatilities = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    log_ret = np.log(df_company_prices['Close'] / df_company_prices['Close'].shift(1))
    volatility = log_ret.rolling(window=252).std() * np.sqrt(252)
    df_company_prices.insert(2, "Volatility", volatility, True)
    l_volatilities.append(df_company_prices)

print(l_volatilities)

### Expected returns

In [ ]:
l_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    returns = (df_company_prices['Close'] - df_company_prices['Close'].shift(1)) / df_company_prices['Close'].shift(1)
    df_company_prices.insert(2, "returns", returns, True)
    l_returns.append(df_company_prices)

In [ ]:
l_returns

In [ ]:
l_yearly_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    try:
        df_yearly = df_company_prices["Close"].resample('Y').first().pct_change()
        
        yearly_mean = df_yearly.mean()
    except:
        print( company)
        print(df_company_prices)
        yearly_mean = 0
    l_yearly_returns.append([company,yearly_mean])
print(l_yearly_returns)

## Prices - BUY

In [ ]:
#price when bought

l_buy_prices=list()
l_volatility=list()
for index,row in transactions.iterrows():
    comp=row['company']
    buy_date=row['date_BUY_fix']
    company_pos=companies.index(comp)
    try:
        price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
        volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
    except :
        price=np.NaN

    l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


In [ ]:
l_volatility_buy=list()
for index,row in transactions.iterrows():
    comp=row['company']
    buy_date=row['date_BUY_fix']
    company_pos=companies.index(comp)
    try:
        volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
    except :
        volatility=np.NaN
    
    l_volatility_buy.append(volatility)
    
print(len(l_volatility_buy))


transactions['Volatility_Buy'] = l_volatility_buy

## Dates - SELL

In [ ]:
#random sell date
dates_SELL=list()
expected_return = list()

for index,row in transactions.iterrows():
    comp=row['company']
    company_pos=companies.index(comp)
    buy_date = row['date_BUY_fix']
    sell_date = buy_date + datetime.timedelta(row['horizon (days)'])
    expected_return.append(row['horizon (days)']/365 * l_yearly_returns[company_pos][1])

    dates_SELL.append(sell_date)

transactions['expected_return (yearly)'] = expected_return
transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [ ]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [ ]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

### Sharpe Ratio 

In [ ]:
l_sharpe=list()
for index,row in transactions.iterrows():
    comp=row['company']
    buy_date=row['date_BUY_fix']
    company_pos=companies.index(comp)
    r = l_returns
    try:
        df_company_returns = pd.DataFrame(l_returns[company_pos])
        r=df_company_returns["returns"]
        sr = r.mean()/r.std() * np.sqrt(252)
        sr= l_volatilities[company_pos].loc[buy_date, 'Volatility']
    except :
        sr=np.NaN
        
    l_sharpe.append(sr)
    
print(len(l_sharpe))


transactions['Sharpe Ratio'] = l_sharpe

In [ ]:
#checking for holidays (NYSE, NASDAQ is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [ ]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=np.NaN

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


In [ ]:
l_volatility_sell=list()
for index,row in transactions.iterrows():
    comp=row['company']
    sell_date=row['date_SELL_fix']
    company_pos=companies.index(comp)
    try:
        volatility=l_volatilities[company_pos].loc[sell_date,'Volatility']
    except :
        volatility=np.NaN

        
    l_volatility_sell.append(volatility)

transactions['Volatility_sell']=l_volatility_sell

In [ ]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']
transactions=transactions[transactions.holiday_SELL!='Holiday']

# Dataset transactions - review missing values

In [ ]:
transactions

In [ ]:
print(transactions.isnull().values.sum())
transactions.isnull().sum()


In [ ]:
df_review=transactions.copy()
print('original    :',df_review.shape[0])
print('without NAN :',df_review.dropna().shape[0])
print('rows with NAN: ',df_review.shape[0]-df_review.dropna().shape[0])

In [ ]:
transactions=transactions.dropna()

In [ ]:
transactions.shape

Now, getting only relevant columns

In [ ]:
transactions_ready=transactions[['company','sector','horizon (days)','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL','Volatility_Buy','Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)']]

In [ ]:
transactions_ready

# Evaluating investment performance 

## Inflation
As investors, we seek to at least beat the rate of inflation. 

In [ ]:
inflation_ratios=pd.read_csv('ratios_inflation.csv', index_col=False)
# inflation_ratios['Year'] = pd.to_datetime(inflation_ratios['Year'], format='%Y')

inflation_ratios=inflation_ratios.set_index('Year')

# inflation.info()

In [ ]:
inflation_ratios.loc[2017,'Tasa_inflacion']

In [ ]:
l_inflation=list()
for index,row in transactions_ready.iterrows():
    buy_date=row['date_BUY_fix']
    buy_date_str=str(buy_date)
    year=int(buy_date_str[:4])
    try:
        inflation=inflation_ratios.loc[year,'Tasa_inflacion']
    except :
        inflation=np.NaN

    l_inflation.append(inflation)

transactions_ready['inflation']=l_inflation


In [ ]:
transactions_ready.isnull().sum

In [ ]:
transactions_ready

## Real Profit/Loss (Rentabilidad Real)

In [ ]:
    transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']


In [ ]:
transactions_ready

In [ ]:
transactions_ready.isnull().sum()

In [ ]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
    inflation=row['inflation']
    if inflation <0:
        if row['nominal_return']>0:
            inv='GOOD'
            investments.append(inv)
        elif row['nominal_return']<0:
            inv='BAD'
            investments.append(inv)
        else:
            inv='BAD'
            investments.append(inv)
    else:
        if row['nominal_return']>inflation:
            inv='GOOD'
            investments.append(inv)
        elif row['nominal_return']<inflation:
            inv='BAD'
            investments.append(inv)
        else:
            inv='BAD'
            investments.append(inv)
        

transactions_ready['investment']=investments

In [ ]:
transactions_ready.shape

In [ ]:
transactions_ready

In [ ]:
transactions_ready.investment.value_counts()

# Ratios

## ESG Ratios

In [ ]:
len(ESG_ratios_fix)

In [ ]:
companies[3]

In [ ]:
ESG_ratios_fix

In [ ]:
transactions_ready['ESG_ranking']=0

for index, row in transactions_ready.iterrows():
    company=row['company']
    ind_comp=companies.index(company)
    ESG_ratio= ESG_ratios_fix[ind_comp]
    transactions_ready.loc[index, 'ESG_ranking']=ESG_ratio
            

In [ ]:
transactions_ready

In [ ]:
transactions_ready.loc[4,'date_BUY_fix']

In [ ]:
pe_ratios_collection= pickle.load(open('pe_ratios_collection.p', "rb" ))

In [ ]:
valor=pe_ratios_collection['AAPL'][pe_ratios_collection['AAPL']['Date']=='2020-09'].PE_Ratio.values[0]
float(valor)

In [ ]:
#getting nearest date before investment.
def nearest(items, pivot):
    return min([i for i in items if i < pivot], key=lambda x: abs(x - pivot))

In [ ]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=pe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PE_ratio_str=pe_ratios_collection[company][pe_ratios_collection[company]['Date']==date].PE_Ratio.values[0]
            PE_ratio=float(PE_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PE_ratio=np.NaN
            
    except:
        PE_ratio=Decimal('nan')
    
    l_PE_ratios.append(PE_ratio)

In [ ]:
transactions_ready['PE_ratio']=l_PE_ratios

In [ ]:
transactions_ready

## EPS Ratio

In [ ]:
import re

In [ ]:
l_EPS_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']

    try:
        ratios_company=pe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),row['date_BUY_fix'])
    
        try:
            EPS_ratio_str=pe_ratios_collection[company][pe_ratios_collection[company]['Date']==date].TTM_Net_EPS.values[0]
            EPS_ratio_str=re.sub('[%$B]', '',EPS_ratio_str)
        
            EPS_ratio=float(EPS_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            EPS_ratio=np.NaN
            
    except:
        EPS_ratio=np.NaN
    
    l_EPS_ratios.append(EPS_ratio)

In [ ]:
transactions_ready['EPS_ratio']=l_EPS_ratios

## PS Ratio

In [ ]:
ps_ratios_collection= pickle.load(open('ps_ratios_collection.p', "rb" ))


In [ ]:
l_PS_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=ps_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PS_ratio_str=ps_ratios_collection[company][ps_ratios_collection[company]['Date']==date].Price_to_Sales_Ratio.values[0]
            PS_ratio=float(PS_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PS_ratio= np.NaN
            
    except:
        PS_ratio=np.NaN
    
    l_PS_ratios.append(PS_ratio)

    
transactions_ready['PS_ratio']=l_PS_ratios

## PB Ratio

In [ ]:
pb_ratios_collection= pickle.load(open('pb_ratios_collection.p', "rb" ))

In [ ]:
l_PB_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=pb_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            PB_ratio_str=pb_ratios_collection[company][pb_ratios_collection[company]['Date']==date].Price_to_Book_Ratio.values[0]
            PB_ratio=float(PB_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PB_ratio= np.NaN
            
    except:
        PB_ratio=np.NaN
    
    l_PB_ratios.append(PB_ratio)

    
transactions_ready['PB_ratio']=l_PB_ratios

## NetProfitMargin ratio 

In [ ]:
NetProfitMargin_ratios_collection= pickle.load(open('NetProfitMargin_ratios_collection.p', "rb" ))

In [ ]:
l_NetProfitMargin_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=NetProfitMargin_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
        try:                       
            NetProfitMargin_ratio_str=NetProfitMargin_ratios_collection[company][NetProfitMargin_ratios_collection[company]['Date']==date].Net_Margin.values[0]
            NetProfitMargin_ratio_str=re.sub('[%$B]', '',NetProfitMargin_ratio_str)
            NetProfitMargin_ratio=float(NetProfitMargin_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            NetProfitMargin_ratio= np.NaN        
    except:
        NetProfitMargin_ratio=np.NaN
    
    l_NetProfitMargin_ratios.append(NetProfitMargin_ratio)
transactions_ready['NetProfitMargin_ratio']=l_NetProfitMargin_ratios

## Current Ratio


In [ ]:
current_ratios_collection= pickle.load(open('current_ratios_collection.p', "rb" ))
current_ratios_collection['AAPL'].head()

In [ ]:
l_current_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=current_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            current_ratio_str=current_ratios_collection[company][current_ratios_collection[company]['Date']==date].Current_Ratio.values[0]
            current_ratio=float(current_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            current_ratio= np.NaN
            
    except:
        current_ratio=np.NaN
    
    l_current_ratios.append(current_ratio)

    
transactions_ready['current_ratio']=l_current_ratios

## ROA Ratio

In [ ]:
roa_ratios_collection= pickle.load(open('roa_ratios_collection.p', "rb" ))
roa_ratios_collection['AAPL'].head()

In [ ]:
l_roa_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=roa_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            roa_ratio_str=roa_ratios_collection[company][roa_ratios_collection[company]['Date']==date].Return_on_Assets.values[0]
            roa_ratio_str=re.sub('[%$B]','',roa_ratio_str)
            roa_ratio=float(roa_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            roa_ratio= np.NaN
            
    except:
        roa_ratio=np.NaN
    
    l_roa_ratios.append(roa_ratio)

    
transactions_ready['roa_ratio']=l_roa_ratios

## ROE Ratio

In [ ]:
roe_ratios_collection= pickle.load(open('roe_ratios_collection.p', "rb" ))
roe_ratios_collection['AAPL'].head()

In [ ]:
l_roe_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=roe_ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),year)
    
        try:
            roe_ratio_str=roe_ratios_collection[company][roe_ratios_collection[company]['Date']==date].Return_on_Equity.values[0]
            roe_ratio_str=re.sub('[%$B]','',roe_ratio_str)
            roe_ratio=float(roe_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            roe_ratio= np.NaN
            
    except:
        roe_ratio=np.NaN
    
    l_roe_ratios.append(roe_ratio)

    
transactions_ready['roe_ratio']=l_roe_ratios

# Final dataset review 

In [ ]:
transactions_ready

In [ ]:
print(transactions_ready.isnull().values.sum())
transactions_ready.isnull().sum()


In [ ]:
df_review=transactions_ready.copy()
print('original    :',df_review.shape[0])
print('without NAN :',df_review.dropna().shape[0])
print('rows with NAN: ',df_review.shape[0]-df_review.dropna().shape[0])

In [ ]:
transactions_ready=transactions_ready.dropna()
transactions_ready.shape

In [ ]:
transactions_ready.to_csv('transactions_variables.csv')